<a href="https://colab.research.google.com/github/xavirubi/tappx-challenge/blob/master/hiring_project/kw_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U keybert

In [ ]:
pip install -U rake-nltk

In [ ]:
pip install -U yake

In [ ]:
pip install googletrans==3.1.0a0

In [ ]:
pip install -U gensim

In [6]:
import json
from keybert import KeyBERT
from rake_nltk import Rake
import nltk
import yake
from googletrans import Translator
from gensim.models import Word2Vec

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
f_articles = open('articles.json')
f_videos = open('videos.json')

articles_data = json.load(f_articles)
videos_data = json.load(f_videos)

article_ids = list(article_id for article_id in articles_data.keys())
video_ids = list(video_id for video_id in videos_data.keys())

In [21]:
def article_text(id):
	return articles_data[id]["text"]

def article_keywords(id):
	return articles_data[id]["keywords"]

def video_keywords(id):
	return videos_data[id]["keywords"]

In [ ]:
kw_model = KeyBERT()
def keybert_keywords(text):
  return kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 3))

In [ ]:
rake_model = Rake()
def rake_keywords(text):
  kw_extracted = rake_model.extract_keywords_from_text(text)
  return rake_model.get_ranked_phrases_with_scores()

translator = Translator()

In [ ]:
def yake_keywords(text):
  kw_extractor = yake.KeywordExtractor(lan='en', n=3)
  return kw_extractor.extract_keywords(text)

In [ ]:
for id in article_ids:
  print(f'''TEXT FROM ARTICLE {id}:
  {article_text(id)}
''')
  print(f'''KEYWORDS FROM ARTICLE {id}:
  {article_keywords(id)}
''')
#   print(f'''KEYBERT KEYWORDS FROM ARTICLE {id}:
# {keybert_keywords(article_text(id))}

  # print(f'''RAKE KEYWORDS FROM ARTICLE {id}:
# {rake_keywords(article_text(id))}

  print(f'''YAKE KEYWORDS FROM ARTICLE {id}:
{yake_keywords(article_text(id))}



-----------------------------------------------------------------------------------------------------
''')

f_articles.close()

In [22]:
corpus1 = article_keywords(article_ids[0])
# corpus2 = article_keywords(article_ids[7])
# print(f"CORPUS 1: {corpus1}\n\nCORPUS 2: {corpus2}\n--------------------------------------------------------------\n\n")

# model = Word2Vec(corpus1, window=5, min_count=1, workers=4)

model = Word2Vec(corpus1)

In [24]:
for i in range(0, len(video_ids)):
  corpus2 = video_keywords(video_ids[i])
  similarity_score = model.wv.n_similarity(corpus1, corpus2)
  print(f"SIMILARITY SCORE: {similarity_score}\n")

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

SIMILARITY SCORE: 0.0

